## Import Module

In [15]:
import pandas as pd
import numpy as np
import optuna
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from scipy.stats import rankdata, pearsonr
from scipy.optimize import minimize
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, Lipinski, Crippen
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['RDK_ERROR_STREAM'] = '/dev/null'
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
optuna.logging.set_verbosity(optuna.logging.WARNING)

## feature cal

In [16]:
def calculate_advanced_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        
        features = {}
        
        try:
            features['MolWt'] = Descriptors.MolWt(mol)
            features['LogP'] = Descriptors.MolLogP(mol)
            features['TPSA'] = Descriptors.TPSA(mol)
            features['NumRotatableBonds'] = Descriptors.NumRotatableBonds(mol)
            features['NumHAcceptors'] = Descriptors.NumHAcceptors(mol)
            features['NumHDonors'] = Descriptors.NumHDonors(mol)
            features['NumAromaticRings'] = Descriptors.NumAromaticRings(mol)
            features['RingCount'] = Descriptors.RingCount(mol)
            features['NumHeteroatoms'] = Descriptors.NumHeteroatoms(mol)
            features['HeavyAtomCount'] = Descriptors.HeavyAtomCount(mol)
        except:
            pass
        
        try:
            features['BertzCT'] = Descriptors.BertzCT(mol)
            features['Chi0'] = Descriptors.Chi0(mol)
            features['Chi1'] = Descriptors.Chi1(mol)
            features['HallKierAlpha'] = Descriptors.HallKierAlpha(mol)
            features['Kappa1'] = Descriptors.Kappa1(mol)
            features['Kappa2'] = Descriptors.Kappa2(mol)
            features['FractionCsp3'] = Descriptors.FractionCsp3(mol)
            features['NumSaturatedRings'] = Descriptors.NumSaturatedRings(mol)
            features['NumAliphaticRings'] = Descriptors.NumAliphaticRings(mol)
            features['MolMR'] = Crippen.MolMR(mol)
            features['BalabanJ'] = Descriptors.BalabanJ(mol)
        except:
            pass
        
        try:
            features['PEOE_VSA1'] = Descriptors.PEOE_VSA1(mol)
            features['PEOE_VSA2'] = Descriptors.PEOE_VSA2(mol)
            features['SMR_VSA1'] = Descriptors.SMR_VSA1(mol)
            features['SlogP_VSA1'] = Descriptors.SlogP_VSA1(mol)
            features['EState_VSA1'] = Descriptors.EState_VSA1(mol)
        except:
            pass
        
        try:
            features['QED'] = Descriptors.qed(mol)
        except:
            pass
        
        try:
            features['NumHeavyAtoms'] = Lipinski.NumHeavyAtoms(mol)
            features['NumAliphaticCarbocycles'] = Lipinski.NumAliphaticCarbocycles(mol)
            features['NumAliphaticHeterocycles'] = Lipinski.NumAliphaticHeterocycles(mol)
            features['NumAromaticCarbocycles'] = Lipinski.NumAromaticCarbocycles(mol)
            features['NumAromaticHeterocycles'] = Lipinski.NumAromaticHeterocycles(mol)
            features['NumSaturatedCarbocycles'] = Lipinski.NumSaturatedCarbocycles(mol)
            features['NumSaturatedHeterocycles'] = Lipinski.NumSaturatedHeterocycles(mol)
        except:
            pass
        
        try:
            features['NumRadicalElectrons'] = Descriptors.NumRadicalElectrons(mol)
            features['NumValenceElectrons'] = Descriptors.NumValenceElectrons(mol)
            
            features['FlexibilityIndex'] = features.get('NumRotatableBonds', 0) / max(features.get('HeavyAtomCount', 1), 1)
            features['TPSARatio'] = features.get('TPSA', 0) / max(features.get('MolWt', 1), 1)
            features['AromaticRatio'] = features.get('NumAromaticRings', 0) / max(features.get('RingCount', 1), 1) if features.get('RingCount', 0) > 0 else 0
            features['HeteroatomRatio'] = features.get('NumHeteroatoms', 0) / max(features.get('HeavyAtomCount', 1), 1)
            
            violations = 0
            if features.get('MolWt', 0) > 500: violations += 1
            if features.get('LogP', 0) > 5: violations += 1
            if features.get('NumHDonors', 0) > 5: violations += 1
            if features.get('NumHAcceptors', 0) > 10: violations += 1
            features['LipinskiViolations'] = violations
        except:
            pass
        
        return features if features else None
        
    except Exception as e:
        return None

## morgan fingerprint

In [17]:
def get_morgan_fingerprint_features(smiles, radius=2, n_bits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(n_bits)
    
    try:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=n_bits)
        return np.array(fp)
    except Exception as e:
        return np.zeros(n_bits)

## Optuna

In [18]:
def create_objective_v2(model_type, X_train, y_train, cv_folds=5):
    
    def objective(trial):
        if model_type == 'lgb':
            params = {
                'objective': 'regression',
                'metric': 'rmse',
                'verbosity': -1,
                'n_estimators': 800,
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'max_depth': trial.suggest_int('max_depth', 3, 15),
                'num_leaves': trial.suggest_int('num_leaves', 20, 400),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 150),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 15.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 15.0),
                'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 1.0),
            }
            model_class = lgb.LGBMRegressor
            
        elif model_type == 'xgb':
            params = {
                'objective': 'reg:squarederror',
                'n_estimators': 800,
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'max_depth': trial.suggest_int('max_depth', 3, 15),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 15.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 15.0),
                'gamma': trial.suggest_float('gamma', 0.0, 8.0),
            }
            model_class = xgb.XGBRegressor
            
        elif model_type == 'catboost':
            params = {
                'iterations': 200,
                'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.3, log=True),
                'depth': trial.suggest_int('depth', 4, 8),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
                'verbose': False,
                'thread_count': 4,
                'random_seed': 42,
            }
            model_class = cb.CatBoostRegressor
            
        elif model_type == 'rf':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 200, 800),
                'max_depth': trial.suggest_int('max_depth', 8, 35),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 8),
                'max_features': trial.suggest_float('max_features', 0.4, 1.0),
                'n_jobs': -1,
                'random_state': 42,
            }
            model_class = RandomForestRegressor
        
        cv = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
        rmse_list = []
        
        for train_idx, val_idx in cv.split(X_train):
            X_fold_train = X_train[train_idx]
            X_fold_val = X_train[val_idx]
            y_fold_train = y_train.iloc[train_idx] if hasattr(y_train, 'iloc') else y_train[train_idx]
            y_fold_val = y_train.iloc[val_idx] if hasattr(y_train, 'iloc') else y_train[val_idx]
            
            model = model_class(**params)
            model.fit(X_fold_train, y_fold_train)
            
            preds = model.predict(X_fold_val)
            rmse = np.sqrt(mean_squared_error(y_fold_val, preds))
            rmse_list.append(rmse)
        
        return np.mean(rmse_list)
    
    return objective

## ensemble blending

In [19]:
def quantile_match(source_pred, target_pred):
    sorted_target = np.sort(target_pred)
    source_ranks = rankdata(source_pred, method='ordinal') - 1
    source_ranks = np.clip(source_ranks, 0, len(sorted_target)-1).astype(int)
    return sorted_target[source_ranks]

def advanced_ensemble_blend(predictions_dict, weights=None):
    if weights is None:
        weights = np.ones(len(predictions_dict)) / len(predictions_dict)
    
    weighted_avg = np.zeros(len(list(predictions_dict.values())[0]))
    for i, pred in enumerate(predictions_dict.values()):
        weighted_avg += weights[i] * pred
    
    ranked_preds = {}
    for name, pred in predictions_dict.items():
        ranked_preds[name] = rankdata(pred) / len(pred)
    
    avg_ranks = np.zeros(len(list(predictions_dict.values())[0]))
    for i, pred in enumerate(ranked_preds.values()):
        avg_ranks += weights[i] * pred
    
    base_pred = list(predictions_dict.values())[0]
    sorted_base = np.sort(base_pred)
    rank_indices = (avg_ranks * (len(sorted_base) - 1)).astype(int)
    rank_indices = np.clip(rank_indices, 0, len(sorted_base) - 1)
    rank_avg = sorted_base[rank_indices]
    
    final_blend = 0.7 * weighted_avg + 0.3 * rank_avg
    
    return final_blend

## data load

In [20]:
df_train = pd.read_csv("/data2/project/2025summer/jjh0709/git/Jump-AI-2025/data/chembl_processed_rescaled.csv")
df_test = pd.read_csv("/data2/project/2025summer/jjh0709/git/Jump-AI-2025/data/test.csv")

df_train = df_train[df_train["IC50"] > 0].copy()
df_train = df_train[(df_train["IC50"] >= 0.1) & (df_train["IC50"] <= 1e5)].copy()
df_train["pIC50"] = 9 - np.log10(df_train["IC50"])

smiles_col = 'Smiles' if 'Smiles' in df_train.columns else 'smiles'
smiles_col_test = 'Smiles' if 'Smiles' in df_test.columns else 'smiles'

## feature extractor

In [21]:
train_features_list = []
for idx, smiles in enumerate(df_train[smiles_col]):
    if idx % 200 == 0:
        print(f"  처리 중: {idx}/{len(df_train)}")
    features = calculate_advanced_features(smiles)
    if features:
        train_features_list.append(features)
    else:
        train_features_list.append({})

train_features_df = pd.DataFrame(train_features_list)
print(f"분자 기술자 완료: {train_features_df.shape}")

  처리 중: 0/806
  처리 중: 200/806
  처리 중: 400/806
  처리 중: 600/806
  처리 중: 800/806
분자 기술자 완료: (806, 29)


## PCA

In [22]:
n_fp_bits = 1024
train_fp_array = np.array([get_morgan_fingerprint_features(s, n_bits=n_fp_bits) 
                          for s in df_train[smiles_col]])

pca = PCA(n_components=100, random_state=42)
train_fp_pca = pca.fit_transform(train_fp_array)
train_fp_df = pd.DataFrame(train_fp_pca, columns=[f'FP_PC{i+1}' for i in range(100)])

print(f"Morgan Fingerprint 완료: {train_fp_df.shape}")

Morgan Fingerprint 완료: (806, 100)


## Feature concat

In [23]:
X_full = pd.concat([train_features_df, train_fp_df], axis=1)
y_full = df_train["pIC50"]

X_full = X_full.fillna(X_full.median())
valid_mask = ~(X_full.isnull().any(axis=1) | y_full.isnull())
X_clean = X_full[valid_mask]
y_clean = y_full[valid_mask]

## Multi-Scaling

In [24]:
scalers = {
    'standard': StandardScaler(),
    'robust': RobustScaler(),
    'quantile': QuantileTransformer(output_distribution='normal', random_state=42)
}

X_scaled = {}
for name, scaler in scalers.items():
    X_scaled[name] = scaler.fit_transform(X_clean)

X_train, X_val, y_train, y_val = train_test_split(
    X_scaled['robust'], y_clean, test_size=0.2, random_state=42
)

print(f"학습/검증 분할 완료: {X_train.shape[0]} / {X_val.shape[0]}")

학습/검증 분할 완료: 644 / 162


## Model clean

In [25]:
best_params = {}
studies = {}

for model_type in ['lgb', 'xgb', 'rf']:
    print(f"  {model_type.upper()} 최적화")
    study = optuna.create_study(direction='minimize')
    study.optimize(
        create_objective_v2(model_type, X_train, y_train),
        n_trials=30,
        show_progress_bar=False
    )
    
    best_params[model_type] = study.best_params
    studies[model_type] = study
    print(f"    Best RMSE: {study.best_value:.4f}")


  LGB 최적화
    Best RMSE: 0.9162
  XGB 최적화
    Best RMSE: 0.9173
  RF 최적화
    Best RMSE: 0.9033


## train model

In [26]:
models = {}

models['lgb'] = lgb.LGBMRegressor(**best_params['lgb'], n_estimators=1200, verbosity=-1)
try:
    models['lgb'].fit(X_train, y_train, 
                      eval_set=[(X_val, y_val)],
                      callbacks=[lgb.early_stopping(80, verbose=False)])
except:
    models['lgb'].fit(X_train, y_train)

models['xgb'] = xgb.XGBRegressor(**best_params['xgb'], n_estimators=1200)
try:
    models['xgb'].set_params(early_stopping_rounds=80)
    models['xgb'].fit(X_train, y_train,
                      eval_set=[(X_val, y_val)],
                      verbose=False)
except:
    try:
        models['xgb'].fit(X_train, y_train,
                          eval_set=[(X_val, y_val)],
                          early_stopping_rounds=80,
                          verbose=False)
    except:
        models['xgb'].fit(X_train, y_train)

models['rf'] = RandomForestRegressor(**best_params['rf'])
models['rf'].fit(X_train, y_train)

models['extra'] = ExtraTreesRegressor(n_estimators=600, max_depth=25, random_state=42, n_jobs=-1)
models['extra'].fit(X_train, y_train)

models['mlp'] = MLPRegressor(
    hidden_layer_sizes=(256, 128, 64),
    activation='relu',
    solver='adam',
    learning_rate='adaptive',
    max_iter=1200,
    early_stopping=True,
    validation_fraction=0.1,
    random_state=42
)
models['mlp'].fit(X_train, y_train)

print("모든 모델 학습 완료오")

모든 모델 학습 완료오


## evaluate

In [27]:
val_predictions = {}
val_scores = {}

for name, model in models.items():
    pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    r2 = r2_score(y_val, pred)
    corr, _ = pearsonr(y_val, pred)
    
    val_predictions[name] = pred
    val_scores[name] = {'rmse': rmse, 'r2': r2, 'corr': corr}
    
    print(f"  {name:10s}: RMSE={rmse:.4f}, R²={r2:.4f}, Corr={corr:.4f}")

  lgb       : RMSE=0.8587, R²=0.3449, Corr=0.5888
  xgb       : RMSE=0.8653, R²=0.3348, Corr=0.5799
  rf        : RMSE=0.8954, R²=0.2877, Corr=0.5475
  extra     : RMSE=1.0330, R²=0.0519, Corr=0.4537
  mlp       : RMSE=1.4678, R²=-0.9141, Corr=0.3368


## ensemble weighted clean

In [28]:
def ensemble_objective(weights):
    ensemble_pred = np.zeros(len(y_val))
    for i, name in enumerate(models.keys()):
        ensemble_pred += weights[i] * val_predictions[name]
    
    rmse = np.sqrt(mean_squared_error(y_val, ensemble_pred))
    return rmse

constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
bounds = [(0, 1) for _ in range(len(models))]
initial_weights = np.ones(len(models)) / len(models)

result = minimize(ensemble_objective, initial_weights, 
                 method='SLSQP', bounds=bounds, constraints=constraints)

optimal_weights = result.x
print(f"최적 가중치:")
for name, weight in zip(models.keys(), optimal_weights):
    if weight > 0.01:
        print(f"  {name}: {weight:.3f}")

최적 가중치:
  lgb: 0.725
  xgb: 0.275


## retrain

In [29]:
models_full = {}

for name in models.keys():
    if name == 'lgb':
        lgb_params = {k: v for k, v in best_params['lgb'].items()}
        models_full[name] = lgb.LGBMRegressor(**lgb_params, n_estimators=1500, verbosity=-1)
    elif name == 'xgb':
        xgb_params = {k: v for k, v in best_params['xgb'].items()}
        models_full[name] = xgb.XGBRegressor(**xgb_params, n_estimators=1500)
    elif name == 'rf':
        rf_params = {k: v for k, v in best_params['rf'].items()}
        models_full[name] = RandomForestRegressor(**rf_params)
    elif name == 'extra':
        models_full[name] = ExtraTreesRegressor(n_estimators=800, max_depth=30, random_state=42, n_jobs=-1)
    elif name == 'mlp':
        models_full[name] = MLPRegressor(
            hidden_layer_sizes=(256, 128, 64),
            activation='relu',
            max_iter=1500,
            random_state=42
        )
    
    models_full[name].fit(X_scaled['robust'], y_clean)

## Test Feature extractor

In [30]:
test_features_list = []
for idx, smiles in enumerate(df_test[smiles_col_test]):
    if idx % 50 == 0:
        print(f"  처리 중: {idx}/{len(df_test)}")
    features = calculate_advanced_features(smiles)
    if features:
        test_features_list.append(features)
    else:
        test_features_list.append({})

test_features_df = pd.DataFrame(test_features_list)

test_fp_array = np.array([get_morgan_fingerprint_features(s, n_bits=n_fp_bits) 
                          for s in df_test[smiles_col_test]])
test_fp_pca = pca.transform(test_fp_array)
test_fp_df = pd.DataFrame(test_fp_pca, columns=[f'FP_PC{i+1}' for i in range(100)])

X_test_full = pd.concat([test_features_df, test_fp_df], axis=1)
X_test_full = X_test_full.fillna(X_test_full.median())

missing_cols = set(X_clean.columns) - set(X_test_full.columns)
for col in missing_cols:
    X_test_full[col] = 0

X_test_full = X_test_full[X_clean.columns]
X_test_scaled = scalers['robust'].transform(X_test_full)

  처리 중: 0/127
  처리 중: 50/127
  처리 중: 100/127


## Predicted test data

In [31]:
test_predictions = {}
for name, model in models_full.items():
    test_predictions[name] = model.predict(X_test_scaled)
    print(f"  {name} 예측 완료")

  lgb 예측 완료
  xgb 예측 완료
  rf 예측 완료
  extra 예측 완료
  mlp 예측 완료


## ensemble blending

In [32]:
ensemble_pred = advanced_ensemble_blend(test_predictions, optimal_weights)

base_pred = test_predictions['rf']
matched_predictions = {}

for name in test_predictions.keys():
    matched_predictions[name] = quantile_match(test_predictions[name], base_pred)

matched_ensemble = advanced_ensemble_blend(matched_predictions, optimal_weights)

final_pred = 0.6 * ensemble_pred + 0.4 * matched_ensemble

## submit submission

In [33]:
final_pred = np.clip(final_pred, y_clean.min(), y_clean.max())
ic50_pred = 10 ** (9 - final_pred)
ic50_pred = np.clip(ic50_pred, 0.1, 100000)

output_dir = "/data2/project/2025summer/jjh0709/git/Jump-AI-2025/submissions/"
os.makedirs(output_dir, exist_ok=True)

submission = pd.DataFrame({
    "ID": df_test["ID"],
    "ASK1_IC50_nM": ic50_pred
})

submission.to_csv(output_dir + "submit_conservative_enhanced.csv", index=False)

ensemble_strategies = {
    'weighted_only': ensemble_pred,
    'quantile_matched': matched_ensemble,
    'final_meta': final_pred
}

for strategy_name, pred in ensemble_strategies.items():
    pred_clipped = np.clip(pred, y_clean.min(), y_clean.max())
    ic50_strategy = 10 ** (9 - pred_clipped)
    ic50_strategy = np.clip(ic50_strategy, 0.1, 100000)
    
    submission_strategy = pd.DataFrame({
        "ID": df_test["ID"],
        "ASK1_IC50_nM": ic50_strategy
    })
    
    filename = f"submit_conservative_{strategy_name}.csv"
    submission_strategy.to_csv(output_dir + filename, index=False)
    print(f"  {filename} 저장 완료")

  submit_conservative_weighted_only.csv 저장 완료
  submit_conservative_quantile_matched.csv 저장 완료
  submit_conservative_final_meta.csv 저장 완료
